In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 GCP 上进行端到端机器学习：MLOps 阶段 6：使用 Vertex AI 在线预测开始使用 AutoML 图像模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK创建图像分类模型，并使用Vertex AI的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML图像分类模型，然后使用Vertex AI SDK进行在线预测。您还可以选择使用`gcloud`命令行工具或在Cloud Console上线创建和部署模型。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AutoML`
- `Vertex AI Prediction`
- `Vertex AI Datasets`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练一个`AutoML`图像分类模型。
- 进行在线预测。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在一个公共的云存储桶中。训练好的模型可以预测一幅图像中的花卉属于五种花卉中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

成本

本教程使用 Google Cloud 的付费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 的价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 的价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)基于您预测的使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex Workbench AI笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- 虚拟环境
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell的命令行中运行`jupyter notebook`。

6. 在Jupyter笔记本仪表板中打开此笔记本。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

### 重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google 云项目

**无论您使用的是笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下方的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 将带有 `!` 前缀的行视为 shell 命令，并且它会插入以 `$` 前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域
您还可以更改“REGION” 变量，该变量用于笔记本的其余操作。以下是 Vertex AI 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶用于 Vertex AI 的训练。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，您可能正在使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在此教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入名称，然后点击**创建**。

在**授予此服务账户对项目的访问**部分，点击角色下拉列表。在筛选框中输入"Vertex"，然后选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，然后选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到本地环境。

将服务账户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入到下面的单元格中，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您在Python中初始化Vertex AI SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在不同会话中保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称在所有Google Cloud项目中必须是全局唯一的，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过查看其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

设置变量

接下来设置一些在整个教程中使用的变量。
导入库并定义常量。

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器镜像和分配给虚拟机（VM）实例的GPU数量。例如，要使用具有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器镜像，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`以使用在CPU上运行的容器镜像。

了解有关[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

了解有关[按机器类型的GPU兼容性](https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table)。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行培训和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

## AutoML 图像模型训练

现在你已经准备好开始创建自己的 AutoML 图像分类模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的Flowers数据集的版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练AutoML模型，您需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
   - `classification`：图像分类模型。
   - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则为单标签（`False`）或多标签（`True`）。
- `model_type`：部署的模型类型。
   - `CLOUD`：在Google Cloud上部署。
   - `CLOUD_HIGH_ACCURACY_1`：针对在Google Cloud上部署时的准确性进行优化。
   - `CLOUD_LOW_LATENCY_`：针对在Google Cloud上部署时的延迟进行优化。
   - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
   - `MOBILE_TF_HIGH_ACCURACY_1`：针对在边缘设备上部署时的准确性进行优化。
   - `MOBILE_TF_LOW_LATENCY_1`：针对在边缘设备上部署时的延迟进行优化。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习--仅支持图像分类。

实例化的对象是训练作业的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + UUID,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练管道

接下来，通过调用`run`方法，并传入以下参数来运行DAG，启动训练任务：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则训练可能在服务认为无法进一步提高模型目标测量之前完成，并且没有使用整个预算。

完成`run`方法后会返回`Model`资源。

训练管道的执行可能需要长达20分钟。

*注意：* 与训练自定义模型不同，经过训练的AutoML模型对应的serving二进制文件是预定义的。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 创建`Endpoint`资源

您可以使用`Endpoint.create()`方法来创建`Endpoint`资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（地区）；否则，设置将继承您在使用`init()`方法初始化Vertex AI SDK时设置的值。

在这个示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的人类可读名称。
- `project`：您的项目ID。
- `location`：您的地区。
- `labels`：（可选的）以键/值对形式提供给`Endpoint`的用户定义的元数据。

此方法返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)的信息。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="flowers_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点。每个被部署的 `Vertex AI Model` 资源都将拥有自己的用于服务二进制文件的部署容器。

*备注:* 对于这个例子，您的 `AutoML` 模型的部署容器是预定义的。

### 缩放

一个 `Vertex AI Endpoint` 资源支持三种类型的缩放：

- 无缩放: 服务二进制文件部署到单个虚拟机实例。
- 手动缩放: 服务二进制文件部署到固定数量的多个虚拟机实例。
- 自动缩放: 服务二进制文件的部署数量会根据负载的情况变化。

对于 AutoML 模型，仅支持无缩放和手动缩放。

### 手动扩展

在下一个示例中，您部署`Vertex AI Model`资源到`Vertex AI Endpoint`资源进行手动扩展--一个固定数量（大于1）的VM实例。换句话说，当模型被部署时，固定数量的VM实例将被配置并保持配置，直到模型被取消部署。

在此示例中，您使用最少数量的指定参数部署模型，如下所示：

- `model`：要部署的`Model`资源。
- `machine_type`：每个VM实例的机器类型。
- `deployed_model_displayed_name`：部署的模型实例的可读名称。
- `min_replica_count`：要配置的最小VM实例（节点）数。
- `max_replica_count`：要配置的最大VM实例（节点）数。
- `accelerator_type`：每个VM实例的硬件加速器类型（如果有）。
- `accelerator_count`：每个VM实例的硬件加速器数量（如果有）。

对于手动扩展，模型部署过程中会配置固定数量的VM实例。

*注意：*对于手动扩展，节点的最小和最大数量设置为相同值。

In [ ]:
MIN_NODES = 2
MAX_NODES = 2

if DEPLOY_NGPU:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="flowers_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="flowers_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

发送一个在线预测请求

向您部署的模型发送在线预测。

获取测试项目

您将使用数据集中的任意示例作为测试项目。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过将预测请求发送到 Endpoint 资源来进行在线预测。

#### 请求

在这个示例中，因为您的测试项目位于一个云存储存储桶中，您可以使用 `tf.io.gfile.GFile()` 打开并读取图片的内容。为了将测试数据传递给预测服务，您需要将字节编码成 base64 格式 -- 这样在通过网络传输二进制数据时可以确保内容不会被修改。

每个实例的格式为：

    { 'content': { 'b64': base64编码的字节 } }

因为 `predict()` 方法可以接受多个项目（实例），所以请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从 `predict()` 调用中返回的响应是一个Python字典，其中包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类标签的类名。
- `confidences`: 每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`: 执行预测的部署的 Model 资源的 Vertex AI 标识符。

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

取消部署模型

当您完成预测后，您可以取消从`Endpoint`资源中部署的模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除模型

接下来，您可以使用`delete()`方法删除`Vertex AI Model`资源。

In [ ]:
model.delete()

### 删除数据集

接下来，您可以使用`delete()`方法删除`Vertex AI Dataset`资源。

In [ ]:
dataset.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的个别资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}